In [ ]:
# 콜레스테롤 예측 모델(실패)



In [1]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

In [2]:
# 1. CSV 파일 불러오기
file_path = "./data/health_2023_cleaned_final.csv"
df = pd.read_csv(file_path, encoding='utf-8', sep=',')

In [3]:
df = pd.read_csv('./data/health_2023_cleaned_final.csv', encoding='utf-8')
df = df[['성별코드', '연령대코드(5세단위)', '신장(5cm단위)', '체중(5kg단위)', '허리둘레', '총콜레스테롤']].dropna()

In [5]:
X = df.drop(columns='총콜레스테롤').values
y = df['총콜레스테롤'].values

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)  # (샘플 수, 타임스텝, 채널)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
# 4. MLP 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # 회귀 출력
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 5. 학습
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

Epoch 1/50
6655/6655 [==============================] - 9s 1ms/step - loss: 2609.0779 - mae: 37.4561 - val_loss: 1601.2620 - val_mae: 31.6426
Epoch 2/50
6655/6655 [==============================] - 14s 2ms/step - loss: 1618.6748 - mae: 31.8472 - val_loss: 1597.3904 - val_mae: 31.6067
Epoch 3/50
6655/6655 [==============================] - 10s 2ms/step - loss: 1614.7631 - mae: 31.7862 - val_loss: 1600.7378 - val_mae: 31.5603
Epoch 4/50
6655/6655 [==============================] - 9s 1ms/step - loss: 1613.4094 - mae: 31.7671 - val_loss: 1592.8247 - val_mae: 31.5673
Epoch 5/50
6655/6655 [==============================] - 20s 3ms/step - loss: 1612.8270 - mae: 31.7610 - val_loss: 1605.3839 - val_mae: 31.5441
Epoch 6/50
6655/6655 [==============================] - 21s 3ms/step - loss: 1612.2943 - mae: 31.7506 - val_loss: 1602.6879 - val_mae: 31.7638
Epoch 7/50
6655/6655 [==============================] - 7s 1ms/step - loss: 1612.8912 - mae: 31.7675 - val_loss: 1614.2174 - val_mae: 31.9010
Ep

KeyboardInterrupt: 

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

# 1. 데이터 불러오기
df = pd.read_csv("./data/health_2023_cleaned_final.csv")

# 2. 파생 변수 생성
df['BMI'] = (df['체중(5kg단위)'] + 2.5) / ((df['신장(5cm단위)'] + 2.5) / 100) ** 2
df['허리신장비'] = df['허리둘레'] / (df['신장(5cm단위)'] + 2.5)

# 3. 결측치 제거
required_cols = [
    '연령대코드(5세단위)', '성별코드', '신장(5cm단위)', '체중(5kg단위)', '허리둘레',
    '흡연상태', '음주여부', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)',
    'BMI', '허리신장비', 'HDL콜레스테롤'
]
df = df.dropna(subset=required_cols)

# 4. 입력 / 출력 정의
X = df[[
    '연령대코드(5세단위)', '성별코드', '신장(5cm단위)', '체중(5kg단위)', '허리둘레',
    '흡연상태', '음주여부', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)',
    'BMI', '허리신장비'
]]
y = np.log1p(df['HDL콜레스테롤'])  # 로그 변환 적용

# 5. 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 6. 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 7. 모델 정의 및 학습
model = XGBRegressor(n_estimators=200, max_depth=4, learning_rate=0.05, random_state=42)
model.fit(X_train, y_train)

# 8. 예측 및 성능 평가
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)  # 로그 복원
y_true = np.expm1(y_test)

print(f"[HDL콜레스테롤 예측 결과 - XGBoost]")
print(f"R²: {r2_score(y_true, y_pred):.4f}")


[HDL콜레스테롤 예측 결과 - XGBoost]
R²: 0.2078


In [39]:
import tensorflow as tf
print(tf.__version__)  # 2.15.1 이 출력돼야 함
print(dir(tf))   

AttributeError: module 'tensorflow' has no attribute '__version__'

In [27]:
# 5. 예측 및 성능 평가
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("📊 MAE:", mae)
print("📊 MSE:", mse)
print("📈 R²:", r2)

📊 MAE: 29.45768543144051
📊 MSE: 1367.5591059781013
📈 R²: 0.2218973049054076
